# Xifrat i probabilitats

En aquest notebook volem abordar una tasca molt concreta: descodificar codis secrets. 

Ho farem amb el més simple dels codis, un xifrat de rotació, de vegades anomenat xifratge de majúscules o xifratge Cèsar (perquè es tractava d’última generació de criptografia al 100 aC). 



In [2]:
from __future__ import division
import re
import math
import string
import random
from collections import Counter
from math import log10
from matplotlib.pyplot import yscale, xscale, title, plot
import requests

alphabet = 'abcdefghijklmnopqrstuvwxyz' 

Per fer-ho ens basarem en les probabilitat de les paraules que poden apareixer en el text. 

Començarem llegint un text molt gran per estimar-les.

In [3]:
r = requests.get('https://raw.githubusercontent.com/norvig/pytudes/master/data/text/big.txt')
r.text #Contnent of the response, in unicode -> r.encoding is utf-8
TEXT = r.text #Cannot print TEXT -> We are exceeding NotebookApp.iopub_data_rate_limit 
print('Tenim un text amb',len(TEXT),'paraules')

Tenim un text amb 6488666 paraules


Anem ara a veure quants *tokens* (o paraules úniques) hi ha i quantes vegades surt cada paraula.

In [4]:
def tokens(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower()) #Look for all regular expressions at the text that use characters.
#With the + we are getting a list of every different word instead a list of every character -> We are not taking upper case letters
#so we need to use text.lower() method

WORDS = tokens(TEXT)
print('Tenim un text amb',len(WORDS),'paraules diferents') 

COUNTS = Counter(WORDS) #We are getting a list of tuples with [('word_n', number of coincidences),...]

# Vegades que surten les 10 paraules més freqüents
print(COUNTS.most_common(10))

# Vegades que surten les paraules d'un text
for w in tokens('the rare and neverbeforeseen words'):
    print(COUNTS[w], w)

Tenim un text amb 1105285 paraules diferents
[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]
80030 the
83 rare
38313 and
0 neverbeforeseen
460 words


Anem ara a calcular la distribució de probabilitats de les paraules i d'una frase formada per aquestes paraules.

In [5]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(counter.values())
    return lambda x: counter[x]/N #For every value on counts (x) calcule its probability on the text

Pword = pdist(COUNTS)

# Imprimim les probabilitats d'unes quantes paraules
for w in tokens('"The" is most common word in English'):
    print('La P de', w, 'és', Pword(w))

def Pwords(words):
    "Probability of a sentence, assuming each word is independent of others."
    if product(Pword(w) for w in words) == 0.0: return None #If the probability of a word is equal to 0 return None
    else: return product(Pword(w) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

print("Probabilitat d'una frase:",Pwords(['large', 'and', 'insignificant']))

La P de the és 0.07240666434449033
La P de is és 0.008842968103249388
La P de most és 0.0008215075749693518
La P de common és 0.0002596615352601365
La P de word és 0.0002696137195383996
La P de in és 0.019949605757790978
La P de english és 0.00019090098933759167
Probabilitat d'una frase: 4.111418791681202e-10


Per últim, anem a definir l'esquema de codificació.

In [6]:
def upperlower(text): return text.upper() + text.lower()  

def rot(msg, n=13): 
    "Encode a message with a rotation (Caesar) cipher." 
    return encode(msg, alphabet[n:]+alphabet[:n])

def encode(msg, key): 
    "Encode a message with a substitution cipher." 
    table = str.maketrans(upperlower(alphabet), upperlower(key))
    return msg.translate(table) 

msg = 'Who knows the answer this time?'
secret = rot(msg, 12)

print('Text:       ',msg)
print('Text xifrat:',secret)


Text:        Who knows the answer this time?
Text xifrat: Ita wzaie ftq mzeiqd ftue fuyq?


## Exercici: 

Completa el codi de la funció 'decode' seguint les següents idees:

+ El xifrat de la frase només pot tenir tantes claus diferents com lletres diferents hi ha.
+ Si provem tots els possibles desxifrats possibles, el correcte és el que dona la seqüencia de paraules més probable.

Tingues en compte també que potser cal modificar la funció `Pword` de manera que cap paraula tingui probabilitat 0.

In [7]:
"""
The algorithm will proceed as the following way:
    1) First we will use a loop to try all the possible keys
    2) For each one of the keys we will calculate the probability of every sentence. We will use a dictionary
       in order to put {probability: key}
    3) Finally we will pick the key which give the greater probability
    4) We will return the decoded message with the key we will try
    
    So this function return a decoded message with the key used on Caesar cipher
    
    :param secret: string with the encoded message
    :return : tuple(string with the decoded message, the key integer)
"""
def decode(secret):
    aux_dict = {} 
    for i in range(len(alphabet)): #Trying the probability of every message and storing results on the dictionary
        words = tokens(rot(secret,i))
        if Pwords(words) != None: aux_dict[Pwords(words)] = i #Pwords modified before to return None instead 0
            
    key = max(aux_dict.values()) #Pick the maximum probability key
    return (rot(secret,n=key), key)

In [8]:
msg = 'Who knows the answer this time?'
secret = rot(msg, 12)

print(secret)
print(decode(secret))


msg = 'Are you ok?'
secret = rot(msg, 17)

print(secret)
print(decode(secret))

Ita wzaie ftq mzeiqd ftue fuyq?
('Who knows the answer this time?', 14)
Riv pfl fb?
('Are you ok?', 9)
